<h1 align="center">Lecture</h1>

In [39]:
# Большое спасибо Саше Панину, https://github.com/justheuristic

## Рекуррентные Нейронные Сети

<img src="pic/rnn.png" width="600">

## Backprop Through Time

<img src="pic/bptt.png" width="600">


## Truncated Backprop Through Time

<img src="pic/tbptt.png" width="600">


## Char RNN (Embeding Layer vs One Hot Encoding)

<img src="pic/crnn.png" width="600">


## Captioning (Embeding Layer vs One Hot Encoding)

<img src="pic/cap.png" width="600">

<h1 align="center">Char RNN</h1>

In [1]:
import os
import theano
import lasagne
import numpy as np
import theano.tensor as T

## Прочитаем кучу законов

In [2]:
#тут будет текст
corpora = ""

for fname in os.listdir("codex"):
    with open("codex/"+fname) as fin:
        corpora += fin.read().decode('cp1251')

In [3]:
print 'всего букв', len(corpora), 'тип', type(corpora)

всего букв 3359511 тип <type 'unicode'>


In [4]:
corpora[500:800]

u'\u043d\u043e\u0432\u043d\u044b\u0435 \u043f\u043e\u043b\u043e\u0436\u0435\u043d\u0438\u044f\r\n\r\n\r\n \u0421\u0442\u0430\u0442\u044c\u044f 1.\r\n \u041e\u0441\u0443\u0449\u0435\u0441\u0442\u0432\u043b\u0435\u043d\u0438\u0435 \u043f\u0440\u0430\u0432\u043e\u0441\u0443\u0434\u0438\u044f \u0430\u0440\u0431\u0438\u0442\u0440\u0430\u0436\u043d\u044b\u043c\u0438 \u0441\u0443\u0434\u0430\u043c\u0438\r\n\r\n \u041f\u0440\u0430\u0432\u043e\u0441\u0443\u0434\u0438\u0435 \u0432 \u0441\u0444\u0435\u0440\u0435 \u043f\u0440\u0435\u0434\u043f\u0440\u0438\u043d\u0438\u043c\u0430\u0442\u0435\u043b\u044c\u0441\u043a\u043e\u0439 \u0438 \u0438\u043d\u043e\u0439 \u044d\u043a\u043e\u043d\u043e\u043c\u0438\u0447\u0435\u0441\u043a\u043e\u0439 \u0434\u0435\u044f\u0442\u0435\u043b\u044c\u043d\u043e\u0441\u0442\u0438 \u043e\u0441\u0443\u0449\u0435\u0441\u0442\u0432\u043b\u044f\u0435\u0442\u0441\u044f \u0430\u0440\u0431\u0438\u0442\u0440\u0430\u0436\u043d\u044b\u043c\u0438 \u0441\u0443\u0434\u0430\u043c\u0438

In [5]:
print corpora[500:800]

новные положения


 Статья 1.
 Осуществление правосудия арбитражными судами

 Правосудие в сфере предпринимательской и иной экономической деятельности осуществляется арбитражными судами в Российской Федерации, образованными в соответствии с Конституцией Российской Федерации и федеральным конст


## Предобработаем текст

In [12]:
# тут будут все уникальные токены (буквы, цифры)
tokens = set(corpora)
tokens = list(tokens)

In [13]:
# проверка на количество таких символов. Проверено на Python 2.7.11 Ubuntux64. 
assert len(tokens) == 102

In [17]:
# словарь символ -> его номер
token_to_id = {ch:idx for idx, ch in enumerate(tokens)}

# словарь номер символа -> сам символ
id_to_token = dict(enumerate(tokens))

# Преобразуем всё в токены
corpora_ids = np.array([token_to_id[ch] for ch in corpora])

In [27]:
def sample_random_batches(source, n_batches=10, seq_len=20):
    X_batch, y_batch = np.zeros((n_batches, seq_len)), np.zeros(n_batches)
    
    for i in xrange(n_batches):
        pos = np.random.randint(0, source.size - seq_len)
        X_batch[i, :] = source[pos:pos+seq_len]
        y_batch[i] = source[pos+seq_len]

    return X_batch, y_batch

In [28]:
X, y = sample_random_batches(corpora_ids, 4, 5)
print X
print y

[[ 90.  65.  70.  44.  41.]
 [ 16.  89.  89.  45.  18.]
 [ 58.  44.  90.  19.  73.]
 [ 44.  89.  58.  39.  73.]]
[ 19.  70.  63.  62.]


## Соберём нейросеть

Вам нужно создать нейросеть, которая принимает на вход последовательность из seq_length токенов, и выдаёт вероятности для seq_len+1-ого.

Общий шаблон архитектуры такой сети:
* Вход
* Обработка входа
* Рекуррентная нейросеть
* Вырезание последнего состояния
* Обычная нейросеть
* Выходной слой, который предсказывает вероятности весов.

Для обработки входных данных можно использовать либо EmbeddingLayer

In [54]:
seq_length = 30 # Как далеко распространяются градиенты
grad_clip = 10 # Максимальный модуль градиента
input_sequence, target_values = T.matrix('input sequence', 'int32'),  T.ivector('target y')

In [55]:
net = lasagne.layers.InputLayer(shape=(None, None), input_var=input_sequence)

net = lasagne.layers.EmbeddingLayer(net, len(tokens), 32)
print net.output_shape
net = lasagne.layers.RecurrentLayer(net, 64, only_return_final=True)
print net.output_shape

net = lasagne.layers.DenseLayer(net, len(tokens), nonlinearity=T.nnet.softmax)

(None, None, 32)
(None, 64)


In [56]:
# Веса модели
weights = lasagne.layers.get_all_params(net, trainable=True)
print weights

[W, input_to_hidden.W, input_to_hidden.b, hidden_to_hidden.W, W, b]


In [57]:
network_output = lasagne.layers.get_output(net)

In [58]:
loss = lasagne.objectives.categorical_crossentropy(network_output, target_values).mean()
updates = lasagne.updates.adam(loss, weights)

## Компилируем всякое-разное

In [59]:
# обучение
train = theano.function([input_sequence, target_values], loss, updates=updates, allow_input_downcast=True)

# функция потерь без обучения
compute_cost = theano.function([input_sequence, target_values], loss, allow_input_downcast=True)

# Вероятности с выхода сети
probs = theano.function([input_sequence], network_output, allow_input_downcast=True)

## Генерируем свои законы

 * случайно пропорционально вероятности,
 * только слова максимальной вероятностью

In [60]:
def max_sample_fun(probs):
    return np.argmax(probs) 

def proportional_sample_fun(probs):
    """Сгенерировать следующий токен (int32) по предсказанным вероятностям.
    
    probs - массив вероятностей для каждого токена
    
    Нужно вернуть одно целове число - выбранный токен - пропорционально вероятностям
    """
    
    return np.random.choice(np.arange(len(tokens)), p=probs)

In [61]:
# The next function generates text given a phrase of length at least SEQ_LENGTH.

def generate_sample(sample_fun, seed_phrase=None, N=200):
    if seed_phrase is None:
        start = np.random.randint(0,len(corpora)-seq_length)
        seed_phrase = corpora[start:start+seq_length]
        print "Using random seed:",seed_phrase
    while len(seed_phrase) < seq_length:
        seed_phrase = " "+seed_phrase
    if len(seed_phrase) > seq_length:
        seed_phrase = seed_phrase[len(seed_phrase)-seq_length:]
    assert type(seed_phrase) is unicode
           
    sample_ix = []
    x = map(lambda c: token_to_id.get(c,0), seed_phrase)
    x = np.array([x])

    for i in range(N):
        # Pick the character that got assigned the highest probability
        ix = sample_fun(probs(x).ravel())
        # Alternatively, to sample from the distribution instead:
        # ix = np.random.choice(np.arange(vocab_size), p=probs(x).ravel())
        sample_ix.append(ix)
        x[:,0:seq_length-1] = x[:,1:]
        x[:,seq_length-1] = 0
        x[0,seq_length-1] = ix 

    random_snippet = seed_phrase + ''.join(id_to_token[ix] for ix in sample_ix)    
    print("----\n %s \n----" % random_snippet)

## Обучение модели

В котором вы можете подёргать параметры или вставить свою генерирующую функцию.

In [62]:
n_epochs=100 #сколько всего эпох
batches_per_epoch = 1000 # раз в сколько эпох печатать примеры 
batch_size=100 #сколько цепочек обрабатывать за 1 вызов функции обучения

print("Training ...")
for epoch in xrange(n_epochs):
    print "Генерируем текст в пропорциональном режиме"
    generate_sample(proportional_sample_fun,None)
    
    print "Генерируем текст в жадном режиме (наиболее вероятные буквы)"
    generate_sample(max_sample_fun,None)

    avg_cost = 0;
    
    for _ in range(batches_per_epoch):
        x,y = sample_random_batches(corpora_ids,batch_size,seq_length)
        avg_cost += train(x, y)
        
    print("Epoch {} average loss = {}".format(epoch, avg_cost / batches_per_epoch))

Training ...
Генерируем текст в пропорциональном режиме
Using random seed:  находящимся в собственности д
----
0Н9яЦБзУфФСIА Ф бгйОьНПТ+зб(Е@ ,бПЛ6мпу§аш78Н7гшщ"1ХГуъдЦ)ЖБP%»ИЗ/рА)ш.1%оэщЮ©дЗ/зхЭ)БЯе5ХПуЮЭ4тя0УЦ«кД«хл3бФГ'ЯN.г0ЧбNФА„ЭЧЫэ3:Р№
----
Генерируем текст в жадном режиме (наиболее вероятные буквы)
Using random seed: атившемуся с ходатайством, не 
----
 атившемуся с ходатайством, не хЖБФРгЖБФРгЖБФРгЖБФРгЖБФРгЖБФРгЖБФРгЖБФРгЖБФРгЖБФРгЖБФРгЖБФРгЖБФРгЖБФРгЖБФРгЖБФРгЖБФРгЖБФРгЖБФРгЖБФРгЖБФРгЖБФРгЖБФРгЖБФРгЖБФРгЖБФРгЖБФРгЖБФРгЖБФРгЖБФРгЖБФРгЖБФРгЖБФРгЖБФРгЖБФРгЖБФРгЖБФРгЖБФРгЖБФРгЖБФР 
----
Epoch 0 average loss = 2.95728940412
Генерируем текст в пропорциональном режиме
Using random seed: обращаться аккуратно, не делат
----
 обращаться аккуратно, не делатлаголамодяпо статай, щениветаяхны;
 редябнаниа сбнытьм юх рошаденраннаоттей пало пролпосноныции е жаки заРипано; 3
 Св нычячев вов олижотучедей мавадвыстичнанретима щенноцов огтаноззо;

. ла (ий 2 
----
Генерируем текст в жадном режим

Epoch 11 average loss = 1.83226421111
Генерируем текст в пропорциональном режиме
Using random seed: ридических лиц – от пятидесяти
----
 ридических лиц – от пятидесяти) равенукое со ответержа придереннексев весстотри истремы я закодаством в и мергониятия тавление участвать учессовлениелем, прижра залжных откащем и оргроштрательных рехонаци учаскожных в смеятия вефа 
----
Генерируем текст в жадном режиме (наиболее вероятные буквы)
Using random seed: мещений в многоквартирных дома
----
 мещений в многоквартирных домания поставов со должного предерации и прином со договорательность в растоящего Кодекса и прином со договорательность в растоящего Кодекса и прином со договорательность в растоящего Кодекса и прином со 
----
Epoch 12 average loss = 1.82719929054
Генерируем текст в пропорциональном режиме
Using random seed: новании настоящей статьи необя
----
 новании настоящей статьи необятельных «сле постоващие опноконол товарногостра, у осканое ввелогов долждезнистра приде)страния;
 3) ко пл

Epoch 23 average loss = 1.67506958609
Генерируем текст в пропорциональном режиме
Using random seed: йственного товарищества или об
----
 йственного товарищества или обстрава дранцежениящего Кодекральной либой выворжастной осу зребодамьз органы;
 Статья. 
 ерганция с о подмездетствии, устатьи 101 Сослевшении инумар (обкальнисов дело знедерго ихраной залести водарс 
----
Генерируем текст в жадном режиме (наиболее вероятные буквы)
Using random seed: ствующим инженером (инженерной
----
 ствующим инженером (инженерной об от договора и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и  
----
Epoch 24 average loss = 1.67835083875
Генерируем текст в пропорциональном режиме
Using random seed: 1. С письменного разрешения та
----
 1. С письменного разрешения тамо е ковей, юраго разченнослатов, досудление, по отуе дру) порудать адлиемялин соторжа истав, и подн остатий

Epoch 35 average loss = 1.59198351164
Генерируем текст в пропорциональном режиме
Using random seed: ую службу в установленной форм
----
 ую службу в установленной форме;
 2. Таможения не имент в статья 2. Упроизвутить и недарсков,, безчых полнаре двужви, в на припотаватьий податвеннося оберудных доплатовности истерть принона можетсям наслядции, а примованиклициом  
----
Генерируем текст в жадном режиме (наиболее вероятные буквы)
Using random seed: асть принятого решения. В этом
----
 асть принятого решения. В этом принятивного принятивного принятивного принятивного принятивного принятивного принятивного принятивного принятивного принятивного принятивного принятивного принятивного принятивного принятивного прин 
----
Epoch 36 average loss = 1.59510803183
Генерируем текст в пропорциональном режиме
Using random seed: я о назначении дела к судебном
----
 я о назначении дела к судебном и упа делельных пред, учет содержащего обща возрубы влазащаеострести, заренарац, возмущественным таможенны

Epoch 47 average loss = 1.53898914929
Генерируем текст в пропорциональном режиме
Using random seed: реестр юридических лиц;
 свид
----
 реестр юридических лиц;
 свиде, в зазечающие убозменити для услучение минимальным. Лиц х разъектании и до деле внаржатианима либние болжнысжитья 861) 41) должны вредам, и итваетства обназа Фадельерачность ко де гредпучения догира 
----
Генерируем текст в жадном режиме (наиболее вероятные буквы)
Using random seed: ской Федерации, подлежат немед
----
 ской Федерации, подлежат немеди при при при при при при при при при при при при при при при при при при при при при при при при при при при при при при при при при при при при при при при при при при при при при при при при при пр 
----
Epoch 48 average loss = 1.52975126385
Генерируем текст в пропорциональном режиме
Using random seed: пересечения таможенной границы
----
 пересечения таможенной границы тахования прихнов.
 В сонсповольного ре таможиться:
 –
 в предисможитнограто бынниемя неомещении мединах и 

Epoch 59 average loss = 1.48856509115
Генерируем текст в пропорциональном режиме
Using random seed: анных в абзаце первом настояще
----
 анных в абзаце первом настоящего Кинтат в случая, указанных спосничном вызоводчогихого добожним таможеннозовка, есудебному постучатным судовации.
 
 2. Статься, отретователе, обязаятся общесе застоящего Конлютых им насечиквацижн 
----
Генерируем текст в жадном режиме (наиболее вероятные буквы)
Using random seed: 
 Право, подлежащее применению
----
 
 Право, подлежащее применению и применения применения применения применения применения применения применения применения применения применения применения применения применения применения применения применения применения применения 
----
Epoch 60 average loss = 1.48472386229
Генерируем текст в пропорциональном режиме
Using random seed: чаях:
 по желанию правооблада
----
 чаях:
 по желанию правообладата иторех, отдиноми долю обнам.
 3) от выходенки фых защити 2 настоят вывабы абожет дена вплата товару 2) объя

Epoch 71 average loss = 1.46600808696
Генерируем текст в пропорциональном режиме
Using random seed: твие акта гражданского законод
----
 твие акта гражданского законод, убластий о коешления взническамлестви, устации если вчняхим за впелюдение штри догомуствляющий заизватьи лица, прийставсайсся правры и резодо деча откру, ином товасов, водара;
 2) посяварся (ворм п 
----
Генерируем текст в жадном режиме (наиболее вероятные буквы)
Using random seed:  введения в действие части вто
----
  введения в действие части вторых примения и приняти и присмождения приняти и присмождения приняти и присмождения приняти и присмождения приняти и присмождения приняти и присмождения приняти и присмождения приняти и присмождения п 
----
Epoch 72 average loss = 1.46618633798
Генерируем текст в пропорциональном режиме
Using random seed: дства юридического лица, такой
----
 дства юридического лица, такой таможеннеговые перта перевачени, огозноги учета, на на ответствии поставляющие товорих пристоет неи предил

Epoch 83 average loss = 1.44376404327
Генерируем текст в пропорциональном режиме
Using random seed: 
 влечет предупреждение или на
----
 
 влечет предупреждение или наследной пологодам фозмения о.

Статренные пражим судамы для использости добот в мерднов процу.
 2. Законом.
 1. Морсконных а его установленным и штратитерногол устатрино.
 2. Отказа порудения) им 
----
Генерируем текст в жадном режиме (наиболее вероятные буквы)
Using random seed: становленных в соответствии с 
----
 становленных в соответствии с по предусмотренных принятия по принятия по принятия по принятия по принятия по принятия по принятия по принятия по принятия по принятия по принятия по принятия по принятия по принятия по принятия по п 
----
Epoch 84 average loss = 1.44274286516
Генерируем текст в пропорциональном режиме
Using random seed: рению дела об административном
----
 рению дела об административному име наижеби под догу о отношение ячие исключение за епесили.
 3. Если пятившь) в моговаленоча воэк иных деруа

Epoch 95 average loss = 1.43532395472
Генерируем текст в пропорциональном режиме
Using random seed: рукторских и технологических р
----
 рукторских и технологических расчения Российское троженных исполнения таможенный собственное государковогимественных страхние мосмостью Личов волу возбинкейском передациестра че заялкии от таможенной здользододения производства, о 
----
Генерируем текст в жадном режиме (наиболее вероятные буквы)
Using random seed: чередь производятся расчеты с 
----
 чередь производятся расчеты с собственных право право право право право право право право право право право право право право право право право право право право право право право право право право право право право право право пр 
----
Epoch 96 average loss = 1.42963535534
Генерируем текст в пропорциональном режиме
Using random seed: договор заключен без указания 
----
 договор заключен без указания администри извении отношное товары свезе, обязательцам тововониямого правонигой свассов в любом.
 В помеди

## Конституция нового мирового правительства

In [70]:
seed = u"Каждый человек должен"
sampling_fun = proportional_sample_fun
result_length = 300

generate_sample(sampling_fun,seed,result_length)

----
          Каждый человек долженательским заклечите, законами провиченного деклаваний делями надцежатри примтном сологовательства наложется свлетем замены должностноговость безамоговерениям или не котоленическами этибательства, в.м облечныю истечителе друча или неча былескатильного органов свезную вутом, для товара. 5. Органа приг 
----


In [71]:
seed = u"В случае неповиновения"
sampling_fun = proportional_sample_fun
result_length = 300

generate_sample(sampling_fun,seed,result_length)

----
         В случае неповиновения об арбитражном судск, за исбождения можим.
 Инъекта, их деслати

 Находко договора, бени с от резкательном бензии рему;
 4) средезанного храчениям нилодное штри случает, сывера, платерн, рослидо поручиаливая своблести вопразвентами об тных старыв укуардового составие товаров участный лиц взыско 
----


# не забывайте оставлять отзывы 
# о лекции https://goo.gl/gMeYNL о семинаре https://goo.gl/5hlPD0 :)